In [21]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('spark_final_task').getOrCreate()

df = spark.read.csv('web_server_logs.csv', header=True, inferSchema=True)
df.printSchema()
df.show(10)


root
 |-- ip: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- method: string (nullable = true)
 |-- url: string (nullable = true)
 |-- response_code: integer (nullable = true)
 |-- response_size: integer (nullable = true)

+---------------+-------------------+------+------------------+-------------+-------------+
|             ip|          timestamp|method|               url|response_code|response_size|
+---------------+-------------------+------+------------------+-------------+-------------+
| 65.188.134.213|2024-06-16 21:03:58|   PUT|          tag/main|          301|         1240|
|  164.201.196.1|2024-02-07 23:17:32|  POST|   blog/main/posts|          200|         8839|
| 181.111.157.10|2024-05-28 17:13:05|DELETE|               app|          404|         3713|
|  125.3.137.142|2024-02-04 18:15:14|DELETE|      main/explore|          200|         9517|
|160.117.189.238|2024-06-23 01:59:10|  POST|        categories|          301|         4523|
| 202.171.48.11

In [22]:
from pyspark.sql.functions import col, count, filter

top_active_ip = df.groupBy(col('ip')).agg(count(col('ip')).alias('request_count')).orderBy(col('request_count').desc()).limit(10)
top_active_ip.show()

+--------------+-------------+
|            ip|request_count|
+--------------+-------------+
| 73.75.132.220|            2|
| 130.46.57.160|            2|
|59.101.173.112|            1|
| 198.252.23.82|            1|
|167.248.71.230|            1|
|125.35.205.176|            1|
|  9.103.109.51|            1|
|   6.1.135.161|            1|
|75.247.198.187|            1|
| 209.37.34.232|            1|
+--------------+-------------+



In [23]:
count_http = df.groupBy('method').agg(count(col('method')).alias('method_count')).orderBy(col('method_count').desc()).limit(10)
count_http.show()

+------+------------+
|method|method_count|
+------+------------+
|DELETE|       25333|
|   PUT|       25054|
|   GET|       24813|
|  POST|       24800|
+------+------------+



In [24]:
respose_404_cnt = df.filter(col('response_code')=='404').groupBy(col('response_code')).agg(count(col('response_code')).alias('total_response_size')).orderBy(col('total_response_size').desc()).limit(10)
respose_404_cnt.show()

+-------------+-------------------+
|response_code|total_response_size|
+-------------+-------------------+
|          404|              24840|
+-------------+-------------------+



In [26]:
from pyspark.sql.functions import to_date
df_with_date = df.withColumn('date', to_date(col('timestamp')))
df_with_date.show(10)


+---------------+-------------------+------+------------------+-------------+-------------+----------+
|             ip|          timestamp|method|               url|response_code|response_size|      date|
+---------------+-------------------+------+------------------+-------------+-------------+----------+
| 65.188.134.213|2024-06-16 21:03:58|   PUT|          tag/main|          301|         1240|2024-06-16|
|  164.201.196.1|2024-02-07 23:17:32|  POST|   blog/main/posts|          200|         8839|2024-02-07|
| 181.111.157.10|2024-05-28 17:13:05|DELETE|               app|          404|         3713|2024-05-28|
|  125.3.137.142|2024-02-04 18:15:14|DELETE|      main/explore|          200|         9517|2024-02-04|
|160.117.189.238|2024-06-23 01:59:10|  POST|        categories|          301|         4523|2024-06-23|
| 202.171.48.119|2024-11-30 03:18:37|   PUT|               app|          404|         2392|2024-11-30|
| 139.65.197.224|2024-03-17 12:09:03|   PUT|   categories/main|          

In [29]:
responses_by_date = df_with_date.groupBy(col('date')).agg(count(col('method')).alias('total_response_size')).orderBy(col('date')).limit(10)
responses_by_date.show()

+----------+-------------------+
|      date|total_response_size|
+----------+-------------------+
|2024-01-01|                300|
|2024-01-02|                289|
|2024-01-03|                255|
|2024-01-04|                286|
|2024-01-05|                255|
|2024-01-06|                268|
|2024-01-07|                277|
|2024-01-08|                286|
|2024-01-09|                257|
|2024-01-10|                276|
+----------+-------------------+

